In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display
import sys
sys.path.append('../scripts') 
from functions import *
from config import nutrient_info, conversion_factors

In [2]:
df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_7340\2711278687.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')


In [3]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [4]:
df_cream_based_foods = df_food_scaled[df_food_scaled['food_group'] == 'cream_based_foods']
df_cream_based_foods.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
302965,"Oh! HIGH PROTEIN NATURE lactose free, sugar free",80.241377,0.416987,62.349128
308629,Chiefs - Protein Pudding (Coco-Almond),75.223594,1.070352,61.913073
305227,Crème Chantilly Chocolat La Laitière - Nestlé,15.781390,0.939142,60.357962
298443,Nestlé The Dairy Mousse Chantilly Chocolate,14.522616,1.234776,54.332210
293938,MIGROS OH! High protein Erdbeer,1.967678,0.479754,76.842593
...,...,...,...,...
313951,"beleaf - Cheesecake alternative Himbeere, Reis...",0.005534,1.024429,0.553360
313139,Cornetto - Soft Ice cream (Strawberry),0.005443,0.251087,0.544329
309026,Coop Qualité&Prix - Rahmglace Schokolade,0.000000,0.000000,0.000000
313578,Coop Qualité & Prix - Cornet mit Rahmglace Van...,0.000000,0.804755,0.000000


In [5]:
high_qi_cream_based_foods = df_cream_based_foods[df_cream_based_foods['QI'] > 2].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [6]:
high_qi_cream_based_foods[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
302965,"Oh! HIGH PROTEIN NATURE lactose free, sugar free",150.0,80.241377,0.416987,62.349128
308629,Chiefs - Protein Pudding (Coco-Almond),80.0,75.223594,1.070352,61.913073
305227,Crème Chantilly Chocolat La Laitière - Nestlé,90.0,15.781390,0.939142,60.357962
298443,Nestlé The Dairy Mousse Chantilly Chocolate,90.0,14.522616,1.234776,54.332210


In [7]:
ratio_cols = [i for i in high_qi_cream_based_foods.columns if '_ratio_scaled' in i]

df_corr = high_qi_cream_based_foods[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

vitamin_e_activity_eaten_ratio_scaled                       -0.995901
beta_carotene_eaten_ratio_scaled                            -0.995901
iron_eaten_ratio_scaled                                     -0.995741
sodium_eaten_ratio_scaled                                   -0.994947
magnesium_eaten_ratio_scaled                                -0.967145
potassium_eaten_ratio_scaled                                -0.914494
sugar_eaten_ratio_scaled                                    -0.875409
salt_eaten_ratio_scaled                                     -0.587960
fiber_eaten_ratio_scaled                                    -0.564773
fat_eaten_ratio_scaled                                      -0.560305
fatty_acids_saturated_eaten_ratio_scaled                    -0.528310
vitamin_b6_eaten_ratio_scaled                                0.505680
protein_eaten_ratio_scaled                                   0.766273
zinc_eaten_ratio_scaled                                      0.998801
pantothenic_acid_eat

In [8]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_cream_based_foods, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )

    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_cream_based_foods, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [9]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- vitamin b1
- vitamin b6

In [10]:
exclude_list = ['vitamin_b1_eaten', 'vitamin_b6_eaten']
qualifying_nutrients = list(nutrient_info.keys())

for idx, row in high_qi_cream_based_foods.iterrows():

    qi_incl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=[])
    qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    nb_incl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=[]) 
    nb_excl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    print(f"Food item : {row['combined_name']}")
    print(f"QI (incl. all): {qi_incl}")
    print(f"QI (excl. {', '.join(exclude_list)}): {qi_excl}")
    print(f"NB (incl. all): {nb_incl:.2f}")
    print(f"NB (excl. {', '.join(exclude_list)}): {nb_excl:.2f}")
    print("-----------")

Food item : Oh! HIGH PROTEIN NATURE lactose free, sugar free
QI (incl. all): 64.29317827973239
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten): 1.8794301256846
NB (incl. all): 57.98
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten): 54.33
-----------
Food item : Chiefs - Protein Pudding (Coco-Almond)
QI (incl. all): 60.43575989546315
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten): 1.7215809178543484
NB (incl. all): 64.09
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten): 60.97
-----------
Food item : Crème Chantilly Chocolat La Laitière - Nestlé
QI (incl. all): 12.840347152919595
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten): 0.9385044304309975
NB (incl. all): 61.99
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten): 61.82
-----------
Food item : Nestlé The Dairy Mousse Chantilly Chocolate
QI (incl. all): 11.905061970574994
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten): 0.9233050747824412
NB (incl. all): 58.21
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten): 57.81
-----------
